In [7]:
import os, sys
# PWD = os.getenv('PWD')
PWD = os.path.abspath('.')

PROJ_MISSING_MSG = """Set an enviroment variable:\n
`DJANGO_PROJECT=your_project_name`\n
or call:\n
`init_django(your_project_name)`
"""

def init_django(project_name=None):
    os.chdir(PWD)
    project_name = project_name or os.environ.get('DJANGO_PROJECT') or None
    if project_name == None:
        raise Exception(PROJ_MISSING_MSG)
    sys.path.insert(0, os.getenv('PWD'))
    os.environ.setdefault('DJANGO_SETTINGS_MODULE', f'{project_name}.settings')
    os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
    import django
    django.setup()

In [8]:
init_django('django_trip_webserver') #project name

In [9]:
from board.models import Trip, Review, ReviewComment
from main.models import TripCategory, AreaL, AreaM
from user.models import User

In [4]:
trip_category = TripCategory(
    trip_category_name = '명소'
)

trip_category.save()

In [5]:
area_large = AreaL(
    area_l_name='서울'
)

area_large.save()

In [6]:
area_medium = AreaM(
    area_l_id = area_large,
    area_m_name = '종로'
)

area_medium.save()

In [7]:
for i in range(9):
    Trip.objects.create(
        trip_category_id = TripCategory.objects.get(id=1),
        trip_category_detail = '상세한 설명',
        trip_name = f'종로 여행지{i}',
        trip_address = f'{i*534}{i*34643}',
        area_m = AreaM.objects.get(id=1),
        trip_time = '11:00:00',
        trip_phone = f'010-{i}{i}{i}{i}-{i}{i}{i}{i}',
        trip_homepage = f'https://jonglo{i*463}{i*2325}.com',
        is_deleted = False
    )

In [8]:
user = User(
    user_name = '정재민',
    user_password = '1234',
    trip_category_1_id = trip_category,
    trip_category_2_id = trip_category,
    trip_category_3_id = trip_category,
    is_deleted = False
)

user.save()

In [9]:
from django.utils import timezone

for i in range(2):
    for j in range(5):
        Review.objects.create(
            user_id = user,
            trip_id = Trip.objects.get(id = 1),
            review_title = '여기 꼭 가보세요',
            review_content = '무조건 후회 안함 강추',
            review_image1 = '',
            review_image2 = '',
            review_image3 = '',
            review_image4 = '',
            review_image5 = '',
            review_time = timezone.localtime(),
            is_deleted = False
        )

In [11]:
for i in range(20):
    ReviewComment.objects.create(
        user_id = User.objects.get(id = 1),
        review_id = Review.objects.get(id = 1),
        review_comment_content = f'안녕하세요 {i}번째 댓글 도배중입니다',
        review_commnet_time = '2001-01-01',
        is_deleted = False
    )

In [30]:
Review.objects.all()[43].review_image1.url

'/review_image/20200902_115034_1_1l6CNc7.jpg'

In [37]:
Review.objects.all()[5].user_id.user_name

'정재민'